In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
def get_geneWithOnbits(gene_str):
    gene_name = gene_str.split("__")[-2] # record gene name
    onBits = []    
    for x in re.split('\]|\[|,',gene_str.split("__")[-1]):
        try:
            onBits.append(int(x))
        except ValueError:
            continue
    return gene_name,onBits        

In [ ]:
lib_path = "U:/Lab/Bereket_public/MERFISH_tools/gene_panel_information/XXBBL1_300_gene_probe_lib_TwistBioscience_120722.txt"
gene_onbits = {}
for line in open(lib_path,'r'):
    if line.startswith('>'):
        gene,onbits = get_geneWithOnbits(line)
        if gene not in gene_onbits:
            gene_onbits[gene] = onbits
        else:
            gene_onbits[gene] +=onbits
        # make sure the onbits are unique    
        gene_onbits[gene] = list(set(gene_onbits[gene]))     

In [ ]:
# convert to dataframe
barcode_len = 27
gene_barcodes = np.zeros((len(gene_onbits),barcode_len))
for i,gene in enumerate(gene_onbits):
    gene_barcodes[i,np.array(gene_onbits[gene])-1] = 1 # Note: -1 is need because python indexing
                                             # starts from 0, while the index for the panel is 
                                             # starting from 1

In [ ]:
vizgen_codebook = pd.read_csv('U:/Lab/Bereket_public/MERFISH_tools/gene_panel_information/Custom_Mouse300GP01_verified.csv',index_col = 0)
blank_barcodes = vizgen_codebook.filter(like='Blank',axis= 0).values[:,:-1]
blank_index = vizgen_codebook.filter(like='Blank',axis= 0).index # name of blank genes


In [ ]:
codebook = pd.DataFrame( np.vstack((gene_barcodes,blank_barcodes)),
                         index = list(gene_onbits.keys()) + list(blank_index),
                         columns = ['bit'+str(i) for i in range(1,barcode_len+1)])
codebook[vizgen_codebook.columns[-1]] = vizgen_codebook.iloc[:,-1]
codebook.index.name = 'name' # name the index column, this is need in the MERlin package

In [ ]:

codebook

In [ ]:
# save the codebook
code_book_21_bits = pd.concat([codebook.iloc[:,:21],codebook.iloc[:,-1]],axis=1)
code_book_21_bits.to_csv("U:/Lab/Bereket_public/custom_rig_related/Merlin_test/cutom_rig_decoding_test/analysis_parameters/codebooks/custom_rig_300GP_21_bits.csv")

In [ ]:
code_book_21_bits